# MMSB textbook figures

## Chapter 1

### Fig 1.07 Collins toggle switch

For Figures 1.7, 7.13, 7.14, 7.15

In [ ]:
using DifferentialEquations
using ComponentArrays
using UnPack
using Plots
Plots.default(linewidth=2, fmt=:png)

In [ ]:
# Convenience functions
hil(x, k) = x / (x + k)
hil(x, k, n) = hil(x^n, k^n)
exprel(x) = x / expm1(x)

In [ ]:
# Time-dependent inhibitor levels
_i1(t) = ifelse(30 <= t <= 40, 10, 0)
_i2(t) = ifelse(10 <= t <= 20, 10, 0)

function collins!(D, u, p, t)
    @unpack a1, a2, β, γ = p
    @unpack s1, s2 = u
    D.s1 = a1 * hil(1 + _i2(t), s2, β) - s1
    D.s2 = a2 * hil(1 + _i1(t), s1, γ) - s2
    nothing
end

In [ ]:
ps = ComponentArray(a1=3.0, a2=2.5, β=4.0, γ=4.0)
u0 = ComponentArray(s1=0.075, s2=2.5)
tend = 50.0

prob = ODEProblem(collins!, u0, tend, ps)
sol = solve(prob, tstops=[10., 20., 30., 40.])

plot(sol, legend=:right, xlabel = "Time", ylabel="Concentration", title="Figure 1.7")

### Fig 1.09 Hodgkin-Huxley model

In [ ]:
using OrdinaryDiffEq
using ComponentArrays
using UnPack
using Plots
Plots.default(linewidth=2, fmt=:png)

In [ ]:
# Build the model
_istim(t) = ifelse(20 <= t <= 21, -6.6, 0.0) + ifelse(60 <= t <= 61, -6.7, 0.0)

function hh!(D, u, p, t)
    @unpack G_N_BAR, E_N, G_K_BAR, E_K, G_LEAK, E_LEAK, C_M = p
    @unpack v, m, h, n = u
    mα = exprel(-0.10 * (v + 35))
    mβ  = 4.0 * exp(-(v + 60) / 18.0)
    hα  = 0.07 * exp(- ( v + 60) / 20)
    hβ  = 1 / (exp(-(v+30)/10) + 1)
    nα  = 0.1 * exprel(-0.1 * (v+50))
    nβ  = 0.125 * exp( -(v+60) / 80)
    iNa = G_N_BAR * (v - E_N) * (m^3) * h
    iK  = G_K_BAR * (v - E_K) * (n^4)
    iLeak = G_LEAK * (v - E_LEAK)
    iStim = _istim(t)
    D.v = -(iNa + iK + iLeak + iStim) / C_M
    D.m = -(mα + mβ) * m + mα
    D.h = -(hα + hβ) * h + hα
    D.n = -(nα + nβ) * n + nα
    nothing
end

In [ ]:
ps = ComponentArray(
    E_N = 55.0,       # Reversal potential of Na (mV)
    E_K = -72.0,      # Reversal potential of K (mV)
    E_LEAK = -49.0,   # Reversal potential of leaky channels (mV)
    G_N_BAR = 120.0,  # Max. Na channel conductance (mS/cm^2)
    G_K_BAR = 36.0,   # Max. K channel conductance (mS/cm^2)
    G_LEAK = 0.30,    # Max. leak channel conductance (mS/cm^2)
    C_M = 1.0        # membrane capacitance (uF/cm^2))
)
u0 = ComponentArray(v=-59.8977, m=0.0536, h=0.5925, n=0.3192)
tend = 100.0

prob = ODEProblem(hh!, u0, tend, ps)
sol = solve(prob, tstops=[20., 60.])

p1 = plot(sol, idxs=(0, :v),
        ylabel="Membrane potential (mV)", xlabel="",
        legend=false, title="Figure 1.9")
p2 = plot(sol, idxs = [:m, :h, :n], xlabel="")
p3 = plot(_istim, sol.t, xlabel="Time (ms)", ylabel="Current",
        labels="Stimulation")
plot(p1, p2, p3, layout=(3, 1), size=(600, 1200))

## Chapter 2

### Fig 2.04 Exponential decay

In [ ]:
using Plots
Plots.default(linewidth=2, fmt=:png)

plot([t-> 3 * exp(-t) t->3 * exp(-2t) t-> 3 * exp(-3t)], 0.0, 5.0, 
     xlim = (0, 5), ylim=(0, 3.2),
     xlabel="Time", ylabel="Concentration", 
     label = ["exp(-t)" "exp(-2t)" "exp(-3t)"], 
     title= "Figure 2.4"
)

### Fig 2.09 Metabolic network simulation

Numerical simulation of a metabolic network using `Catalyst.jl`

In [ ]:
using DifferentialEquations
using Catalyst
using Plots
Plots.default(linewidth=2, fmt=:png)

In [ ]:
# Model building
rn = @reaction_network begin
    3.0, 0 --> A
    2.0, A --> B
    2.5, A + B --> C + D
    3.0, C --> 0
    3.0, D --> 0
end

In [ ]:
u0 = zeros(4)
tend = 10.0
sol = solve(ODEProblem(rn, u0, tend))

plot(sol, legend=:bottomright, title="Figure 2.09", 
    xlims=(0.0, 4.0), ylims=(0.0, 1.0), 
    xlabel="Time (sec)", ylabel="Concentration (mM)")

### Figure 2.11

Model reduction of ODE metabolic networks

In [ ]:
using DifferentialEquations
using ComponentArrays
using UnPack
using Plots
Plots.default(linewidth=2, fmt=:png)

In [ ]:
function model211!(D, u, p, t)
    @unpack k0, k1, km1, k2 = p
    @unpack a, b = u
    vab = k1 * a - km1 * b
    D.a = k0 - vab
    D.b = vab - k2 * b
end

In [ ]:
ps1 = ComponentArray(k0=0., k1=9., km1 = 12., k2 = 2.)
u0 = ComponentArray(a=0., b=10.)
tend = 3.0

sol211 = solve(ODEProblem(model211!, u0, tend, ps1))

plot(sol211, xlabel="Time (AU)", ylabel="Concentration (AU)", title="Fig. 2.11 (Full model)")

### Figure 2.12 : Rapid equilibrium assumption

In [ ]:
using DifferentialEquations
using ComponentArrays
using UnPack
using Plots
Plots.default(linewidth=2, fmt=:png)

In [ ]:
a212(u, p) = hil(p.km1, p.k1) * u
b212(u, p) = hil(p.k1, p.km1) * u

function model212(u, p, t)
    @unpack k0, k2 = p
    b = b212(u, p)
    return k0 - k2 * b
end

In [ ]:
sol212 = solve(ODEProblem(model212, sum(u0), tend, ps1))

pl212 = plot(sol211, line=(:dash, 1),label=["A (full solution)" "B (full solution)"])

plot!(pl212, t -> a212(sol212(t), ps1), 0, tend, label="A (rapid equilibrium)")
plot!(pl212, t -> b212(sol212(t), ps1), 0, tend, label="B (rapid equilibrium)")
plot!(pl212, title="Fig. 2.12 (Rapid equilibrium model)",
        xlabel="Time (AU)", ylabel="Concentration (AU)")

### Figure 2.13: Rapid equilibrium (take 2)

When another set of parameters is not suitable for rapid equilibrium assumption.

In [ ]:
using DifferentialEquations
using ComponentArrays
using UnPack
using Plots
Plots.default(linewidth=2, fmt=:png)

ps2 = ComponentArray(k0=9., k1=20., km1 = 12., k2 = 2.)
u0 = ComponentArray(a=8., b=4.)
tend = 3.0

sol213full = solve(ODEProblem(model211!, u0, tend, ps2))
sol213re = solve(ODEProblem(model212, sum(u0), tend, ps2))

pl213 = plot(sol213full, line=(:dash, 1), label=["A (full solution)" "B (full solution)"])
plot!(pl213, [t -> a212(sol213re(t), ps2) t -> b212(sol213re(t), ps2)], 0, tend, label=["A (rapid equilibrium)" "B (rapid equilibrium)"])
plot!(pl213, title="Fig. 2.13 (Rapid equilibrium model)", xlabel="Time (AU)", ylabel="Concentration (AU)")

### Figure 2.14 : QSSA

Quasi-steady state assumption on species A

In [ ]:
using DifferentialEquations
using ComponentArrays
using UnPack
using Plots
Plots.default(linewidth=2, fmt=:png)

function model214(b, p, t)
    @unpack k0, k2 = p
    return k0 - b * k2
end

# The initial conditions in QSSA is not trivial
function u0214(u0, p)
    @unpack k0, k1, km1, k2 = p
    return (k1 * sum(u0) - k0) / (k1 + km1)
end

function a214(b, p)
    @unpack k0, k1, km1, k2 = p
    return (k0 + km1 * b)/k1
end

sol214 = solve(ODEProblem(model214, u0214(u0, ps2), tend, ps2))

pl214 = plot(sol213full, line=(:dash), xlims=(0.0, tend),
    xlabel="Time (arbitrary units)",
    ylabel="Concentration (arbitrary units)",
    title="Figure 2.14: Ref vs QSSA")

plot!(pl214, sol214, label="B (QSSA)", line=(:red))
plot!(pl214, t -> a214(sol214(t), ps2), 0, tend, label="A (QSSA)", line=(:blue))

### Problem 2.4.6

In [ ]:
using DifferentialEquations
using Plots
Plots.default(linewidth=2, fmt=:png)

# Model
f(u, p, t) = p * (1.0 - u)

p = 1.0
u0 = 0.0
tspan = 10.0

prob = ODEProblem(f, u0, tspan, p)
sol = solve(prob)

plot(sol)

## Chapter 3

### Figure 3.03 Michaelis-Menten kinetics

In [ ]:
using DifferentialEquations
using ComponentArrays
using UnPack
using Plots
Plots.default(linewidth=2, fmt=:png)

In [ ]:
function model303!(D, u, p, t)
    @unpack k1, km1, k2, et = p
    @unpack s, es, p = u
    e = et - es
    v1 = k1 * s * e - km1 * es
    v2 = k2 * es
    D.s = -v1
    D.es = v1 - v2
    D.p = v2
end

In [ ]:
u0 = ComponentArray(s = 5., es = 0., p = 0.)
ps = ComponentArray(k1 = 30., km1 = 1., k2 = 10., et = 1.)
tend = 1.0

prob = ODEProblem(model303!, u0, tend, ps)
sol = solve(prob)

plot(sol, xlabel="Time (AU)", ylabel="Concentration (AU)", legend=:right)

In [ ]:
function mm303(u, p, t)
    @unpack k1, km1, k2, et = p
    ds = - k2 * et * hil(u, (km1 + k2) / k1)
    return ds
end

In [ ]:
u0simp = sum(u0)
solmm = solve(ODEProblem(mm303, sum(u0), tend, ps))

pl303 = plot(sol, idxs = [:s, :p], line=(:dash), label=["S (full)" "P (full)"])
plot!(pl303, [(t -> solmm(t)) (t -> u0simp - solmm(t))], 0.0, tend, label=["S (MM)" "P (MM)"])
plot!(pl303, title="Fig. 3.03", xlabel="Time (AU)", ylabel="Concentration (AU)", xlims=(0.0, tend), ylims=(0.0, u0simp), legend=:right)

## Chapter 4

### Figure 4.1, 4.2, and 4.3

Steady states and phase plots in an assymetric network.

In [ ]:
using DifferentialEquations
using ComponentArrays
using UnPack
using Plots
using LinearAlgebra
Plots.default(linewidth=2, fmt=:png)

In [ ]:
function model41!(D, u, p, t)
    @unpack k1, k2, k3, k4, k5, n = p
    @unpack a, b = u
    D.a = k1 * hil(1, b, n) - (k3 + k5) * a
    D.b = k2 + k5 * a - k4 * b
    return nothing
end

In [ ]:
ps1 = ComponentArray(k1=20., k2=5., k3=5., k4=5., k5=2., n=4.)
u0s = [
    ComponentArray(a=0.0, b=0.0),
    ComponentArray(a=0.5, b=0.6),
    ComponentArray(a=0.17, b=1.1),
    ComponentArray(a=0.25, b=1.9),
    ComponentArray(a=1.85, b=1.7),
]

tend = 1.5

In [ ]:
sols = map(u0s) do u0
    prob = ODEProblem(model41!, u0, tend, ps1)
    sol = solve(prob)
end

plot(sols[1], xlabel="Time", ylabel="Concentration", title="Fig. 4.2 A (Time series)", labels=["[A]" "[B]"])

In [ ]:
plot(sols[1], idxs=(:a, :b), xlabel="[A]", ylabel="[B]", aspect_ratio=:equal,
     title="Fig. 4.2 B (Phase plot)", ylims=(0.0, 2.0), xlims=(0.0, 2.0), 
     legend=nothing)

In [ ]:
p43a = plot(title="Fig. 4.3A (Multiple time series)")
	
for sol in sols
    plot!(p43a, sol, linealpha=0.5, legend=nothing)
end

plot!(p43a, xlabel="Time", ylabel="Concentration")

In [ ]:
p43b = plot(title="Fig. 4.3 B (Phase plot)")

for sol in sols
    plot!(p43b, sol, idxs=(:a, :b), legend=nothing)
end

plot!(p43b, xlabel="[A]", ylabel="[B]", xlims=(0., 2.), ylims=(0., 2.), aspect_ratio=:equal)

### Figure 4.4 and 4.5 

Vector fields in phase plots.

In [ ]:
# Nullclines
nca404(b, p) = p.k1 / (p.k5 + p.k4) * hil(1, b, p.n)
ncb404(b, p) = (p.k4 * b - p.k2) / p.k5

function ∂F404(x, y, p; scale=20)
    u = ComponentArray(a=x, b=y)
    D = copy(u)
    model41!(D, u, p, nothing)
    return D ./ (norm(D)^0.5 * scale)
end

In [ ]:
# Mesh points
xx = [x for y in 0.0:0.2:2.0, x in 0.0:0.2:2.0];
yy = [y for y in 0.0:0.2:2.0, x in 0.0:0.2:2.0];

In [ ]:
p44a = plot(title="Fig. 4.4 A (Phase plot with vector field)")

for sol in sols
    plot!(p44a, sol, idxs=(:a, :b), linealpha=0.7, legend = nothing)
end

quiver!(p44a, xx, yy, quiver=(x,y)->∂F404(x, y, ps1; scale=20), 
        line=(:lightgrey), aspect_ratio=:equal, arrow=(:closed),
        xlabel="[A]", ylabel="[B]", xlims=(0., 2.), ylims=(0., 2.), size=(600, 600)
)

In [ ]:
# Figure 4.5A
p45a = plot(title="Fig. 4.5 A (Phase plot with nullclines)")

# Phase plots
for sol in sols
    plot!(p45a, sol, idxs=(:a, :b), linealpha=0.7, lab=nothing)
end

# Parametric plots for nullclines
plot!(p45a, b -> nca404(b, ps1), identity, 0., 2., label="A nullcline", line=(:black, :dot))
plot!(p45a, b -> ncb404(b, ps1), identity, 0., 2., label="B nullcline", line=(:black, :dash))
plot!(p45a, xlim=(0., 2.), ylim=(0., 2.), legend=:bottomright, size=(600, 600), xlabel="[A]", ylabel="[B]", aspect_ratio=:equal)

In [ ]:
p45b = quiver(xx, yy, quiver=(x,y)->∂F404(x, y, ps1; scale=20), line=(:lightgray), arrow=(:closed), 
            title="Fig. 4.5 B (Vector field with nullclines)", xlabel="[A]", ylabel="[B]")
plot!(p45b, b -> nca404(b, ps1), identity, 0., 2., label="A nullcline", line=(:black, :dot))
plot!(p45b, b -> ncb404(b, ps1), identity, 0., 2., label="B nullcline", line=(:black, :dash))
plot!(p45b, aspect_ratio=:equal, xlim=(0., 2.), ylim=(0., 2.), legend=:bottomright, size=(600, 600))

### Figure 4.7, 4.8, 4.9, and 4.19A

Symmetric (bistable) biological networks.

In [ ]:
using DifferentialEquations
using ComponentArrays
using UnPack
using Plots
using LinearAlgebra
Plots.default(linewidth=2, fmt=:png)

In [ ]:
function model47!(D, u, p, t)
    @unpack k1, k2, k3, k4, n1, n2 = p
    @unpack a, b = u
    D.a = k1 * hil(1, b, n1) - k3 * a
    D.b = k2 * hil(1, a, n2) - k3 * b
end

In [ ]:
ps1 = ComponentArray(k1=20., k2=20., k3=5., k4=5., n1=4., n2=1.)
tend = 4.0

sol1 = solve(ODEProblem(model47!, ComponentArray(a=3.,b=1.), tend, ps1))
sol2 = solve(ODEProblem(model47!, ComponentArray(a=1.,b=3.), tend, ps1))

p47a1 = plot(sol1, xlabel="Time", ylabel="Concentration", legend=:right, title= "Fig 4.7A (1)")
p47a2 = plot(sol2, xlabel="Time", ylabel="Concentration", legend=:right, title= "Fig 4.7A (2)")
fig47a = plot(p47a1, p47a2, layout=(2, 1), size=(600, 600))

In [ ]:
r = range(0., 5., 21)
xx = [x for y in r, x in r]
yy = [y for y in r, x in r];

In [ ]:
nca47(b, p) = p.k1 / p.k3 * hil(1, b, p.n1)
ncb47(a, p) = p.k2 / p.k4 * hil(1, a, p.n2)

function ∂F47(a, b, p; scale=20)
    u = ComponentArray(a=a, b=b)
    D = copy(u)
    model47!(D, u, p, nothing)
	return D ./ (norm(D)^0.5 * scale)
end

In [ ]:
p47b = quiver(xx, yy, quiver=(x, y)-> ∂F47(x, y, ps1; scale=20), line=(:lightgrey), arrow=(:closed), aspect_ratio=:equal, size =(600, 600))
plot!(p47b, b -> nca47(b, ps1), identity, 0., 5., label="Nullcline A", line=(:dash, :red))
plot!(p47b, identity, a -> ncb47(a, ps1), 0., 5., label="Nullcline B", line=(:dash, :blue))
plot!(p47b, title="Fig 4.7 B", xlim=(0., 5.), ylim=(0., 5.), aspect_ratio=:equal, size =(600, 600), xlabel="[A]", ylabel="[B]")

In [ ]:
ps2 = ComponentArray(k1=20., k2=20., k3=5., k4=5., n1=4., n2=4.)

tend = 4.0
sol1 = solve(ODEProblem(model47!, ComponentArray(a=3.,b=1.), tend, ps2))
sol2 = solve(ODEProblem(model47!, ComponentArray(a=1.,b=3.), tend, ps2))

pl48a1 = plot(sol1, xlabel="Time", ylabel="Concentration", legend=:right, title= "Fig 4.8A (1)")
pl48a2 = plot(sol2, xlabel="Time", ylabel="Concentration", legend=:right, title= "Fig 4.8A (2)")
fig48a = plot(pl48a1, pl48a2, layout=(2, 1), size=(600, 600))

In [ ]:
p48b = quiver(xx, yy, quiver=(x, y)-> ∂F47(x, y, ps2; scale=20), line=(:lightgrey), arrow=(:closed), aspect_ratio=:equal, size =(600, 600))
plot!(p48b, b -> nca47(b, ps2), identity, 0., 5., label="Nullcline A", line=(:dash, :red))
plot!(p48b, identity, a -> ncb47(a, ps2), 0., 5., label="Nullcline B", line=(:dash, :blue))
plot!(p48b, title="Fig 4.8 B", xlim=(0., 5.), ylim=(0., 5.), aspect_ratio=:equal, size =(600, 600), xlabel="[S1]", ylabel="[S2]")

In [ ]:
r2 = range(1.0, 1.5, 11)
xx2 = [x for y in r2, x in r2]
yy2 = [y for y in r2, x in r2]

p48b = quiver(xx2, yy2, quiver=(x, y)-> ∂F47(x, y, ps2; scale=60), line=(:lightgrey), arrow=(:closed), aspect_ratio=:equal, size =(600, 600))
plot!(p48b, b -> nca47(b, ps2), identity, r2[1], r2[end], label="Nullcline A", line=(:dash, :red))
plot!(p48b, identity, a -> ncb47(a, ps2), r2[1], r2[end], label="Nullcline B", line=(:dash, :blue))
plot!(p48b, title="Fig 4.8 B (close up)", xlim=(r2[1], r2[end]), ylim=(r2[1], r2[end]), aspect_ratio=:equal, size =(600, 600), xlabel="[S1]", ylabel="[S2]")

In [ ]:
pls = map((8.0, 16.0, 20.0, 35.0)) do k1
    ps = ComponentArray(k1=k1, k2=20., k3=5., k4=5., n1=4., n2=4.)
    pl = plot(b -> nca47(b, ps), identity, 0., 7., label="Nullcline A")
    plot!(pl, identity, a -> ncb47(a, ps), 0., 7., label="Nullcline B")
    plot!(pl, title = "K1 = $k1", xlim=(0., 7.), ylim=(0., 7.), aspect_ratio = :equal, xlabel="[A]", ylabel="[B]")
    pl
end

plot(pls..., size = (600, 600))

### Figure 4.11

Surface plots reference: [surface plots @ PlotsGallery.jl](https://goropikari.github.io/PlotsGallery.jl/src/surface.html)

In [ ]:
using Plots
Plots.default(linewidth=2, fmt=:png)

z1(x, y) = x^2 + 0.5y^2
z2(x, y) = (.2x^2-1)^2 + y^2
x1 = y1 = range(-1.0, 1.0, 41)
x2 = range(-2.75, 2.75, 41)
y2 = range(-0.75, 0.75, 41)
p1 = surface(x1, y1, z1, title="Single-well potential")
p2 = contourf(x1, y1, z1)
p3 = surface(x2, y2, z2, title="Double-well potential")
p4 = contourf(x2, y2, z2)

plot(p1, p2, p3, p4, size=(800, 600))

### Figure 4.15, 4.16, and 4.17

Oscillatory networks.

In [ ]:
using DifferentialEquations
using ComponentArrays
using LinearAlgebra
using Plots
Plots.default(linewidth=2, fmt=:png)

In [ ]:
function model415!(D, u, p, t)
    @unpack k0, k1, k2, n = p
    @unpack a, b = u
    vab = k1 * a * (1 + b^n)
    D.a = k0 - vab
    D.b = vab - k2 * b
end

In [ ]:
function figure0415(; 
    ps = ComponentArray(k0 = 8., k1 = 1., k2 = 5., n = 2.),
	r = range(0., 4., 21),
	tend = 8.,
	figtitle="Fig 4.15",
    model = model415!,
    u0s = (
        ComponentArray(a=1.5, b=1.0),
        ComponentArray(a=0.0, b=1.0), 
        ComponentArray(a=0.0, b=3.0),
        ComponentArray(a=2.0, b=0.0),
    )
)
	sols = map(u0s) do u0
		solve(ODEProblem(model, u0, tend, ps))
	end

	p1 = plot(sols[1], xlabel="Time", ylabel="Concentration", title ="$figtitle (A)", xlims=(0., 8.))

    function ∂F(x, y; scale=20)
        u = ComponentArray(a=x, b=y)
        D = copy(u)
        model415!(D, u, ps, nothing)
        return D ./ (norm(D)^0.5 * scale)
    end

	nc_a(b) = ps.k0 / ps.k1 * hil(1, b, ps.n)
    nc_b(b) = (ps.k2 * b) / (ps.k1 * (1 + b ^ ps.n))
	
	xx = [x for y in r, x in r]
	yy = [y for y in r, x in r]

	p2 = plot(title = "$figtitle (B)")
	for sol in sols
		plot!(p2, sol, idxs=(:a, :b), label=nothing)
	end
	
	rMin, rMax = r[begin], r[end]
	
	plot!(p2, nc_a, identity, rMin, rMax, label="Nullcline A", line=(:dash, :red))
	plot!(p2, nc_b, identity, rMin, rMax, label="Nullcline B", line=(:dash, :blue))
    quiver!(p2, xx, yy, quiver=∂F, line=(:lightgrey), xlims=(rMin, rMax), ylims=(rMin, rMax), 
        aspect_ratio=:equal, size=(600, 600), arrow=(:closed), xlabel="[A]", ylabel="[B]")
	return (p1, p2)
end

In [ ]:
fig415a, fig415b = figure0415()

In [ ]:
fig415a

In [ ]:
fig415b

In [ ]:
fig416a, fig416b = figure0415(ps = ComponentArray(k0 = 8., k1 = 1., k2 = 5., n = 2.5) , tend = 1000.0, figtitle="Fig 4.16")

In [ ]:
fig416a

In [ ]:
fig416b

In [ ]:
_, fig417 = figure0415(
    ps = ComponentArray(k0 = 8., k1 = 1., k2 = 5., n = 2.5), 
    tend = 10., r = range(0., 4., 21), 
    u0s = ( ComponentArray(a=2.0, b=1.5),),
)

In [ ]:
plot!(fig417, xlims=(1.0, 3.0), ylims=(1.0, 3.0), title="Fig 4.17")

### Figure 4.18 Continuation diagram

See also [BifurcationKit.jl](https://github.com/bifurcationkit/BifurcationKit.jl)

In [ ]:
using DifferentialEquations
using ComponentArrays
using LinearAlgebra
using Plots
Plots.default(linewidth=2, fmt=:png)

In [ ]:
function model418!(D, u, p, t)
    @unpack a, b = u
    @unpack k1, k2, k3, k4, k5, n = p
    D.a = k1 * hil(1, b, n) - (k3 + k5) * a
    D.b = k2 + k5 * a - k4 * b
end

In [ ]:
ps = ComponentArray(k1 = 20.0, k2 = 5.0, k3 = 5.0, k4 = 5.0, k5 = 2.0, n = 4)
u0 = ComponentArray(a=0., b=0.)
tend = 1000.

In [ ]:
# Could also use parallel ensemble: https://diffeq.sciml.ai/stable/features/ensemble/

r = range(0., 1000., 51)

aInf = map(r) do k1
    p = copy(ps)
    p[1] = k1
	prob = ODEProblem(model418!, u0, tend, p)
	sol = solve(prob, Rodas5(), save_everystep=false)
	sol.u[end].a
end

plot(r, aInf, title = "Fig 4.18 Continuation diagram", 
     xlabel = "K1" , ylabel= "Steady state [A]", 
     legend=nothing, ylim=(0, 4), xlim=(0, 1000))

### Figure 4.22 Tangent line

In [ ]:
using Plots
Plots.default(linewidth=2, fmt=:png)

plot(t -> 3 / (t-2), 2.2, 8.0, lab="Curve")
plot!(t -> 1.5 - (t - 4) * 0.75, 2.7, 5.3, lab="Tangent line")
plot!(title="Fig 4.22", xlabel="Reaction rate", ylabel="Inhibitor concentration", 
      xlims=(2., 8.), ylims=(0., 4.))

## Runtime information

In [ ]:
versioninfo()

In [ ]:
using Pkg
Pkg.status()